In [1]:
from astropy.cosmology import Planck13
import pandas as pd
import numpy as np
import disperse
from tqdm import tqdm
import json
%matplotlib inline
import matplotlib.pyplot as plt

In [2]:
cosmo = Planck13

H0 = cosmo.H0.value
Om = cosmo.Om0
Ol = 0.69288
Ok = 0.0

sph2cart = 'dist'
in_cart_coords = True

BOARD = 'smooth'
ANGLE = 30

In [3]:
coma_DR7 = pd.read_csv('coma_DR7_filtered_RaDecZ.csv')
coma_DR12 = pd.read_csv('coma_DR12_filtered_RaDecZ.csv')
rosat = pd.read_csv('rosat_filtered_RaDecZ.csv')
sdss_opt_gr_full = pd.read_csv('sdss_opt_gr_filtered_RaDecZ.csv')

In [4]:
# N_GAL >= 15

In [5]:
min_n_gals = 15

In [6]:
sdss_opt_gr = sdss_opt_gr_full[sdss_opt_gr_full['n_gal'] >= min_n_gals]
sdss_opt_gr.reset_index(drop=True, inplace=True)

In [7]:
DPS = disperse.Disperse3D(
    coma_DR12, sdss_opt_gr, '_disperse_03/bin/',
    H0, Om, Ol, Ok,
    sph2cart_f=sph2cart
)
DPS.count_cart_coords()

CX = DPS.galaxies['CX']
CY = DPS.galaxies['CY']
CZ = DPS.galaxies['CZ']

CX_int = (CX.min(), CX.max())
CY_int = (CY.min(), CY.max())
CZ_int = (CZ.min(), CZ.max())

np.random.seed(0)

random_clusters_num = 5
random_clusters = []
for i in range(random_clusters_num):
    cx = np.random.uniform(CX_int[0], CX_int[1], DPS.clusters.shape[0])
    cy = np.random.uniform(CY_int[0], CY_int[1], DPS.clusters.shape[0])
    cz = np.random.uniform(CZ_int[0], CZ_int[1], DPS.clusters.shape[0])
    df = pd.DataFrame()
    df = df.assign(CX=cx)
    df = df.assign(CY=cy)
    df = df.assign(CZ=cz)
    df = df.assign(R200=DPS.clusters['R200'])
    random_clusters.append(df)

In [8]:
random_clusters[0]

,CX,CY,CZ,R200
0,-84.754430,-80.441688,30.109617,1.794430
1,-55.744866,75.866360,50.748292,1.262930
2,-75.347643,-106.638644,133.815451,1.407870
3,-85.439727,121.981974,164.183359,1.102600
4,-106.577304,42.918725,161.241446,1.072530
...,...,...,...,...
217,-113.585055,-38.328354,141.323229,0.792467
218,-77.866517,-54.513705,9.574399,0.549744
219,-35.543488,84.988491,36.988159,0.569996
220,-70.776149,113.689902,60.591641,0.485982


In [9]:
metrics = {}

In [10]:
RAD_COEFS = list(np.arange(1, 20, 1.0))
metrics['mode'] = 'MPC'
metrics['COEFS'] = RAD_COEFS
metrics['min_n_gals'] = min_n_gals
metrics['total_clusters'] = sdss_opt_gr.shape[0]

In [11]:
for SIGMA in list(np.arange(3, 5.5, 1.0)):
    metrics[SIGMA] = {}
    for SMOOTH in range(0, 3):
        metrics[SIGMA][SMOOTH] = {}
        DPS = disperse.Disperse3D(
            coma_DR12, sdss_opt_gr, '_disperse_03/bin/',
            H0, Om, Ol, Ok,
            sph2cart_f=sph2cart
        )
        DPS.count_cart_coords()
        DPS.apply_disperse(SIGMA, SMOOTH, BOARD, ANGLE, in_cart_coords)
        true_inter = []
        true_fils_inter = []
        for COEF in tqdm(RAD_COEFS):
            cl_conn, cl_maxmap, \
            fils_conn, maxs_maxmap, \
            count_fils, count_maxs = DPS.count_conn_maxmap(
                [COEF] * sdss_opt_gr.shape[0],
                [COEF] * sdss_opt_gr.shape[0]
            )
            true_inter.append(sum(list(map(lambda x: int(x>0), cl_conn))))
            true_fils_inter.append(sum(list(map(lambda x: int(x>0), fils_conn))))
        true_inter = np.array(true_inter)
        true_fils_inter = np.array(true_fils_inter)
        
        false_inter = []
        false_fils_inter = []
        for i in tqdm(range(random_clusters_num)):
            false_inter.append([])
            false_fils_inter.append([])
            for COEF in RAD_COEFS:
                cl_conn, cl_maxmap, \
                fils_conn, maxs_maxmap, \
                count_fils, count_maxs = DPS.count_conn_maxmap(
                    [COEF] * sdss_opt_gr.shape[0],
                    [COEF] * sdss_opt_gr.shape[0],
#                     sdss_opt_gr['R200'] * COEF,
#                     sdss_opt_gr['R200'] * COEF,
                    random_clusters[i]
                )
                false_inter[i].append(sum(list(map(lambda x: int(x>0), cl_conn))))
                false_fils_inter[i].append(sum(list(map(lambda x: int(x>0), fils_conn))))
                
        false_inter = np.array(false_inter).mean(0)
        false_fils_inter = np.array(false_fils_inter).mean(0)
        
        diff = true_inter - false_inter
        fils_diff = true_fils_inter - false_fils_inter
        
        true_inter = [int(e) for e in true_inter]
        false_inter = [float(e) for e in false_inter]
        diff = [float(e) for e in diff]
        true_fils_inter = [int(e) for e in true_fils_inter]
        false_fils_inter = [int(e) for e in false_fils_inter]
        fils_diff = [int(e) for e in fils_diff]
        
        metrics[SIGMA][SMOOTH]['true'] = list(true_inter)
        metrics[SIGMA][SMOOTH]['false'] = list(false_inter)
        metrics[SIGMA][SMOOTH]['diff'] = list(diff)
        metrics[SIGMA][SMOOTH]['fils_num'] = len(DPS.fils)
        metrics[SIGMA][SMOOTH]['true_fils_inter'] = list(true_fils_inter)
        metrics[SIGMA][SMOOTH]['false_fils_inter'] = list(false_fils_inter)
        metrics[SIGMA][SMOOTH]['fils_diff'] = list(fils_diff)

100%|██████████| 5/5 [03:04<00:00, 36.95s/it]


In [12]:
metrics

{'mode': 'MPC',
 'COEFS': [1.0,
  2.0,
  3.0,
  4.0,
  5.0,
  6.0,
  7.0,
  8.0,
  9.0,
  10.0,
  11.0,
  12.0,
  13.0,
  14.0,
  15.0,
  16.0,
  17.0,
  18.0,
  19.0],
 'min_n_gals': 15,
 'total_clusters': 222,
 3.0: {0: {'true': [168,
    201,
    214,
    219,
    220,
    221,
    222,
    222,
    222,
    222,
    222,
    222,
    222,
    222,
    222,
    222,
    222,
    222,
    222],
   'false': [1.6,
    6.8,
    12.2,
    23.2,
    36.8,
    47.8,
    58.6,
    67.8,
    79.0,
    91.0,
    97.8,
    103.8,
    111.6,
    116.4,
    120.8,
    125.2,
    129.4,
    132.0,
    136.0],
   'diff': [166.4,
    194.2,
    201.8,
    195.8,
    183.2,
    173.2,
    163.4,
    154.2,
    143.0,
    131.0,
    124.2,
    118.2,
    110.4,
    105.6,
    101.2,
    96.8,
    92.6,
    90.0,
    86.0],
   'fils_num': 2949,
   'true_fils_inter': [292,
    457,
    591,
    697,
    799,
    930,
    1036,
    1146,
    1251,
    1349,
    1436,
    1544,
    1653,
    1746,
    18

In [13]:
with open("metrics_05.json", "w") as f:
    json.dump(metrics, f)